In [2]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((227, 227)),  
    transforms.Grayscale(num_output_channels=3),  
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [3]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: skashperova (kashperova-test). Use `wandb login --relogin` to force relogin


True

In [4]:
wandb.init(
    project="cv-iasa",  
    name="lab2"       
)

In [5]:
import hydra
from torch import optim, nn

from models import AlexNet
from modules.trainers import BaseTrainer
from utils.metrics import Task, Metrics

In [6]:
num_classes = 10
model = AlexNet(num_classes=num_classes)
metrics = Metrics(task=Task.MULTICLASS_CLASSIFICATION, average="macro", num_classes=num_classes)

In [7]:
hydra.initialize(config_path="../src/config/trainers", version_base=None)
config = hydra.compose(config_name="alexnet.yaml")

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=1, min_lr=1e-6)
loss_fn = nn.CrossEntropyLoss()


In [9]:
trainer = BaseTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    config=config,
    loss_fn=loss_fn,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    metrics=metrics,
    n_splits=5
)

In [10]:
trainer.cross_validate(verbose=True)


Starting fold 1/5


Training:   0%|          | 0/20 [02:54<?, ?it/s]


KeyboardInterrupt: 